## **| 앙상블 학습과 랜덤 포레스트 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch07 앙상블 학습과 랜덤 포레스트 연습문제 2, 7, 8, 9번
- 이론적 지식을 묻는 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

In [1]:
# import libraries
import numpy as np

### **1. 직접 투표와 간접 투표 분류기 사이의 차이점은 무엇일까요?**
___


직접 투표 분류기는 각 분류기의 예측을 모아서 다수결 투표와 같이 가장 많이 선택된 클래스를 예측하는 방식으로, 각 분류기가 약한 학습기일지라도 충분하게 많고 다양하다면 앙상블은 강한 학습기가 될 수 있는 원리를 이용한 것이다. 

간접 투표 분류기는 모든 분류기가 클래스의 확률을 예측할 수 있으면 개별 분류기의 각 클래스의 평균적인 확률 추정값을 계산해서 확률이 가장 높은 클래스를 예측하는 방식이다. 신뢰가 더 높은 투표에 더 가중치를 주고 종종 더 나은 성능을 낸다.

### **2. 그레디언트 부스팅 앙상블이 훈련 데이터에 과대 적합되었다면 학습률을 어떻게 해야 할까요?**
___

학습률을 감소시킨다. 

예측기 수가 너무 많으면 알맞은 개수를 찾기 위해 조기 종료 기법을 이용할 수 있다.

### **3. [실습] 다음 지시에 따라 투표 기반 분류 모델을 만들어 보세요**
___

#### **STEP 1. MNIST 데이터를 불러들이고, 훈련, 검증, 테스트 데이터로 나누세요.**

In [2]:
# import MNIST dataset
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame = False)
X, y = mnist["data"], mnist["target"]

In [4]:
# train/valid/test dataset
from sklearn.model_selection import train_test_split
X_train_val, X_test, y_train_val, y_test = train_test_split(X,y,test_size=10000)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=10000)

####  **STEP 2. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시키세요.**
- 모델 파라미터는 `n_estimators=100`, `random_state=42`로 설정합니다. 

In [5]:
# import package
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [6]:
# model fitting
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
et_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
svm_clf = LinearSVC(max_iter=100, tol=20, random_state=42)
mlp_clf = MLPClassifier(random_state=42)

In [7]:
classifier = [rf_clf, et_clf, svm_clf, mlp_clf]
for model in classifier:
  print("Training the ",model)
  model.fit(X_train, y_train)

Training the  RandomForestClassifier(random_state=42)
Training the  ExtraTreesClassifier(random_state=42)
Training the  LinearSVC(max_iter=100, random_state=42, tol=20)
Training the  MLPClassifier(random_state=42)


In [8]:
[model.score(X_val, y_val) for model in classifier]

[0.9707, 0.9725, 0.859, 0.957]

In [27]:
y_predictions = []
from sklearn.metrics import accuracy_score
for model in classifier:
  y_pred = model.predict(X_val)
  y_predictions.append(accuracy_score(y_val, y_pred))
print(y_predictions)

[0.9707, 0.9725, 0.859, 0.957]


####  **STEP 3-1. 앞에서 훈련시킨 각 모델을 직접 투표 방법을 사용해 앙상블로 연결하고 훈련시킨 후, `score()`메서드를 이용하여 검증 데이터셋에서의 성능을 평가해보세요.**

In [12]:
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(
    estimators = [('rf_clf',rf_clf),('et_clf',et_clf), ('svm_clf',svm_clf), ('mlp_clf',mlp_clf)]
)
# model fitting
voting_clf.fit(X_train, y_train)
# model test
voting_clf.score(X_val, y_val)

0.9697

####  **STEP 3-2. 검증 데이터셋에서 각 분류 모델의 성능을 `score()` 메서드를 이용하여 확인해보고, 가장 성능이 낮은 모델을 제거하여 그 결과를 비교해보세요.**

- Hint : 가장 성능이 낮은 모델을 제거할 때 `del`를 활용해보세요

In [13]:
# 각 분류 모델 학습
voting_clf.estimators_

[RandomForestClassifier(random_state=42),
 ExtraTreesClassifier(random_state=42),
 LinearSVC(max_iter=100, random_state=42, tol=20),
 MLPClassifier(random_state=42)]

In [20]:
[model.score(X_val, y_val) for model in classifier]

[0.9707, 0.9725, 0.859, 0.957]

- Q. 어떤 모델의 성능이 가장 낮나요?
- A. SVM의 LinearSVC 모델

In [21]:
# 가장 성능이 낮은 모델 제거
del voting_clf.estimators_[2]

In [22]:
voting_clf.estimators_

[RandomForestClassifier(random_state=42),
 ExtraTreesClassifier(random_state=42),
 MLPClassifier(random_state=42)]

In [23]:
# 모델 제거 후 성능 확인
voting_clf.score(X_val, y_val)

0.9743

In [24]:
# 간접 투표 분류기 이용
voting_clf.voting = 'soft'
voting_clf.score(X_val, y_val)
# 이 데이터에서는 직접 투표 분류기의 성능이 더 좋음.

0.965

### **4. 다음 단계를 따라 앞에서 훈련시킨 분류 모델들을 이용하여 스태킹 앙상블을 구성해보자.**
___

#### **STEP 1. 3번 문제의 각 분류 모델을 실행해서 검증 세트에서 예측을 만들고, 그 결과로 훈련 세트를 만들어 보세요.**

In [33]:
X_val_predictions = np.empty((len(X_val), len(classifier)), dtype=np.float32)

for index, model in enumerate(classifier):
  X_val_predictions[:,index] = model.predict(X_val)

In [34]:
X_val_predictions

array([[5., 5., 5., 5.],
       [2., 2., 2., 2.],
       [7., 7., 7., 7.],
       ...,
       [8., 8., 9., 8.],
       [5., 5., 5., 5.],
       [4., 4., 4., 4.]], dtype=float32)

####  **STEP 2. 새로운 훈련 세트를 이용하여 랜덤 포레스트 분류 모델을 학습시켜 보세요.**

In [36]:
rf_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rf_blender.fit(X_val_predictions, y_val)

RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)

In [37]:
rf_blender.oob_score_

0.9704

In [39]:
rf_blender.score(X_val_predictions, y_val)

0.9834

- 이 랜덤 포레스트 분류 모델이 바로 블렌더에 해당합니다. 

####  **STEP 3. 이제 테스트셋에서 스태킹 앙상블 모델을 평가해보세요.**
- 성능 평가 지표로 **정확도**를 이용하세요.

In [40]:
# 각 분류 모델의 예측을 만들어 새로운 데이터셋 생성
X_test_predictions = np.empty((len(X_test), len(classifier)), dtype=np.float32)

for index, model in enumerate(classifier):
  X_test_predictions[:,index] = model.predict(X_test)

In [41]:
X_test_predictions

array([[7., 7., 9., 7.],
       [1., 1., 1., 1.],
       [6., 6., 6., 6.],
       ...,
       [1., 1., 1., 1.],
       [3., 3., 3., 3.],
       [5., 5., 1., 5.]], dtype=float32)

In [42]:
# 새로운 데이터셋을 이용하여 블렌더로 예측
y_pred = rf_blender.predict(X_test_predictions)

In [43]:
# model test
accuracy_score(y_pred, y_test)

0.9671

In [46]:
rf_blender.score(X_test_predictions, y_test)

0.9671